In [1]:
import calcbench as cb
from sklearn.feature_extraction.text import TfidfVectorizer
from bs4 import BeautifulSoup
from scipy.spatial.distance import cosine
from IPython.core.display import display, HTML
import sklearn
import itertools
from tqdm import tqdm_notebook
from matplotlib import colors
import matplotlib.pyplot as plt

In [2]:
class NumberNormalizingVectorizer(sklearn.feature_extraction.text.TfidfVectorizer):
    def build_tokenizer(self):
        tokenize = super(NumberNormalizingVectorizer, self).build_tokenizer()
        return lambda doc: list(number_normalizer(tokenize(doc)))

In [3]:
def number_normalizer(tokens):
    """ Map all numeric tokens to a placeholder.

    For many applications, tokens that begin with a number are not directly
    useful, but the fact that such a token exists can be relevant.  By applying
    this form of dimensionality reduction, some methods may perform better.
    """

    return ("#NUMBER" if token[0].isdigit() else token for token in tokens)

In [4]:
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = itertools.tee(iterable)
    next(b, None)
    return zip(a, b)

In [13]:
tickers = """ABCB
ABTX
ACBI
AGM
AMTD
ANCX
AROW
ASB
ATAX
BAC
BANC
BANF
BANR
BBT
BCBP
BCML
BDGE
BFIN
BFST
BGCP
BHB
BHBK
BHLB
BKU
BMRC
BMTC
BNCL
BOCH
BOFI
BOH
BPFH
BPRN
BRKL
BSRR
BUSE
BWB
BXS
BY
C
CAC
CADE
CARO
CASH
CATC
CATY
CBSH
CBTX
CBU
CCBG
CCNE
CFFI
CFG
CFR
CHCO
CHFC
CIT
CIVB
CLBK
CMA
CNBKA
CNOB
COBZ
COLB
COWN
CPF
CSFL
CSTR
CTBI
CUBI
CVBF
CVCY
CZNC
DCOM
EBSB
EFSC
EGBN
EQBK
EQH
ESQ
ETFC
EVR
EWBC
FBC
FBIZ
FBK
FBMS
FBNC
FBNK
FCB
FCBC
FCF
FCNCA
FDEF
FFBC
FFIC
FFIN
FFNW
FFWM
FHB
FHN
FIBK
FISI
FITB
FLIC
FMAO
FMBH
FMBI
FMNB
FNB
FNLC
FRBK
FRC
FRME
FSB
FSBW
FULT
GABC
GBCI
GBNK
GCAP
GHL
GNBC
GNTY
GS
GSBC
GWB
HAFC
HBAN
HBCP
HBMD
HBNC
HFWA
HIFS
HLI
HMST
HOMB
HONE
HOPE
HTBI
HTBK
HTH
HTLF
HWC
IBCP
IBKC
IBKR
IBOC
IBTX
INBK
INDB
INTL
ISBC
ITG
JPM
KEY
KRNY
LBAI
LBC
LEVL
LION
LKFN
LOB
LPLA
LTS
LTXB
MBFI
MBIN
MBTF
MBWM
MC
MCB
MCBC
MOFG
MS
MSBI
MSL
MTB
MTG
NBHC
NBN
NBTB
NCBS
NCOM
NFBK
NMIH
NRIM
NWBI
NYCB
OBNK
OCFC
OCN
OLBK
ONB
OPB
OPY
ORIT
OSBC
OZK
PACW
PB
PBCT
PCSB
PEBO
PFBC
PFS
PFSI
PGC
PHH
PJC
PJT
PNC
PNFP
PPBI
PRK
PUB
PWOD
QCRH
RBB
RBCAA
RDN
RF
RILY
RJF
RNST
RVSB
SASR
SBBX
SBCF
SBNY
SBSI
SBT
SCHW
SF
SFBS
SFNC
SFST
SHBI
SIEB
SIVB
SMBC
SMBK
SMMF
SNFCA
SNV
SONA
SRCE
SSB
STBA
STBZ
STI
STL
STXB
SYBT
TBBK
TBK
TBNK
TCBI
TCBK
TCF
THFF
TMP
TOWN
TREE
TRMK
TRST
TSBK
TSC
UBNK
UBSH
UBSI
UCBI
UCFC
UMBF
UMPQ
USB
UVSP
VBTX
VIRT
VLY
VOYA
WABC
WAFD
WAL
WASH
WBS
WD
WFC
WNEB
WSBC
WSBF
WSFS
WTBA
WTFC
ZION
""".splitlines()

In [15]:
document_section = "Management's Discussion And Analysis"
first_year = 2008
end_year = 2018
diffs = pd.DataFrame(index=tickers, columns=range(end_year, first_year, -1))
for ticker in tqdm_notebook(tickers):
    ten_K_sections = (d for d in cb.document_search(company_identifiers=[ticker], 
                                                document_name=document_section, 
                                                all_history=True) if d['fiscal_period'] == 'Y')
    sorted_disclosures = sorted(ten_K_sections, key=lambda d: d['fiscal_year'])
    year_pairs = pairwise(sorted_disclosures)
    for last_year, this_year in year_pairs:
        text_last_year = BeautifulSoup(last_year.get_contents(), 'html.parser').text
        text_this_year = BeautifulSoup(this_year.get_contents(), 'html.parser').text
        vectorizer = NumberNormalizingVectorizer(stop_words='english')
        X = vectorizer.fit_transform([text_this_year, text_last_year])
        distance = cosine(X[0].todense(), X[1].todense())
        diffs[this_year['fiscal_year']][ticker] = distance

In [6]:
def background_gradient(s, m, M, cmap='PuBu', low=0, high=0):
    # from https://stackoverflow.com/questions/38931566/pandas-style-background-gradient-both-rows-and-columns
    rng = M - m
    norm = colors.Normalize(m - (rng * low),
                            M + (rng * high))
    normed = norm(s.values)
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ['background-color: %s' % color for color in c]

def highlight_largest_diffs(diffs):
    filled_df = diffs.loc[diffs.sum(axis=1).sort_values(ascending=False).index].fillna(0).round(3)
    return filled_df.style.apply(background_gradient, cmap='Reds', m=filled_df.min().min(), M=filled_df.max().max(), low=0, high=2.5)

## Hightlight Risk Factors with Greatest Change
### Brightest cells are those documents which changed the most vis-a-vis the previous period.

In [7]:
highlight_largest_diffs(diffs)

,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009
CVX,0.004,0.006,0,0.023,0.01,0.409,0.945,0.117,0,0
PG,0.005,0.007,0.009,0.01,0.006,0.001,0.471,0.529,0,0
CAT,0.002,0.003,0.002,0.003,0.007,0,0.414,0.43,0.002,0
WBA,0.007,0.005,0.008,0.023,0,0.243,0.391,0.001,0,0
INTC,0.001,0.03,0.015,0.008,0.052,0.007,0.004,0.011,0.006,0
MSFT,0.024,0.007,0.017,0.023,0,0.005,0.004,0.009,0,0
JPM,0.001,0.001,0.001,0.001,0.043,0.035,0.002,0.002,0,0
V,0.004,0.005,0.02,0.014,0.003,0.005,0.011,0.005,0.008,0
AXP,0.004,0.004,0.004,0.006,0.022,0.015,0.006,0.003,0.004,0
DWDP,0.012,0.025,0.003,0.004,0.004,0.003,0.007,0.003,0.006,0


In [12]:
diffs.to_excel("C:\\Users\\Andrew Kittredge\\Dropbox (Calcbench)\\Andrew\\tf-idf examples\\md_a.xlsx")

In [8]:
document_section = "Management's Discussion And Analysis"
ticker = "CVX"
year = 2012
previous_year = 2011
doc = next(cb.document_search(company_identifiers=[ticker], document_name=document_section, year=year)).get_contents()
previous_doc = next(cb.document_search(company_identifiers=[ticker], document_name=document_section, year=previous_year)).get_contents()
display(HTML(cb.html_diff(doc, previous_doc)))

## Review Changes
#### The .607 distance between JNJ's 2015 and 2016 risk factors indicates a substantial change.  We verify the change on Calcbench's [disclosure page](https://www.calcbench.com/query/footnotes?pg_classificationMethod=tickers&pg_tickers=JNJ&doc_searchingBy=footnoteType&doc_footnoteType=1110&doc_selectedDisclosure=b-648365_section111&pc_year=2016&pc_periodType=Annual&pc_useFiscalPeriod=false&pc_rangeOption=Single%20Period&pc_dateRange=%5Bobject%20Object%5D).
![Diff](https://dl.dropboxusercontent.com/s/vjd382gr4vvhvuh/diff.png?raw=1)